In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [5]:
path_fs = os.path.join("tmp_data", "db_fs.csv")
path_fca = os.path.join("tmp_data", "db_fca.csv")
path_faue = os.path.join("tmp_data", "db_faue.csv")

# db_fs find march churn uids

In [6]:
df_fs = pd.read_csv(path_fs)
df_fs.head(3)

,record_id,event_created_at_utc,start_date,end_date,sub_plan_id,sub_status_id,payment_number,sub_type_id,paid_period_days,merchant_id,...,usd_revenue,profit,cancelled_at,refund_price,ab_test_id,ab_group_id,app_id,platform_id,user_had_trial,autorenew_off_at
0,1999146,2021-12-08 15:48:02,2021-12-08 11:00:00,2021-12-15 11:00:00,30,3,0,3,7,1,...,0,0,NaN,0,28,6,3.0,2,True,2021-12-15 06:00:00
1,1999196,2021-12-09 04:00:09,2021-12-09 00:00:00,2022-01-09 00:00:00,33,2,21,3,31,1,...,799,679,NaN,0,28,6,2.0,2,True,NaN
2,2000623,2021-12-13 02:40:26,2021-12-12 22:00:00,2022-01-12 22:00:00,33,2,5,3,31,1,...,799,559,NaN,0,28,6,2.0,2,False,NaN


In [7]:
g_uid_fs = df_fs.groupby("user_id")

buff = (g_uid_fs.end_date.last() < "2022-01-01 23:59:59") & (g_uid_fs.start_date.last() > "2021-12-01 00:00:00")
buff


user_id
63         False
71         False
136        False
190        False
208        False
           ...  
2260411    False
2260417    False
2260430    False
2260454    False
2260483    False
Length: 112658, dtype: bool

In [8]:
dec_active_jan_churn = buff.index[buff == True]
dec_active_jan_churn


Int64Index([ 122044,  755215, 1008731, 1098977, 1225931, 1265602, 1322489,
            1450346, 1459667, 1522815,
            ...
            2230067, 2230081, 2230106, 2230111, 2230170, 2230563, 2230720,
            2231422, 2232083, 2232832],
           dtype='int64', name='user_id', length=620)

In [9]:
churned_in_jan = pd.Series(dec_active_jan_churn)
churned_in_jan.to_csv('churned_in_jan.csv', header= False, index= False)

# -----------------------------------------------------------------

In [10]:
buff = (g_uid_fs.end_date.last() > "2022-01-01 23:59:59")
buff

user_id
63         False
71         False
136        False
190        False
208        False
           ...  
2260411     True
2260417     True
2260430     True
2260454     True
2260483     True
Name: end_date, Length: 112658, dtype: bool

In [11]:
usrs_need_to_drop = buff.index[buff == True]
usrs_need_to_drop

Int64Index([    439,    4300,    4534,    8207,   39533,   45211,   53235,
              54376,   61380,   77230,
            ...
            2260297, 2260311, 2260317, 2260364, 2260392, 2260411, 2260417,
            2260430, 2260454, 2260483],
           dtype='int64', name='user_id', length=5899)

In [12]:
buff = pd.Series(usrs_need_to_drop)
buff.to_csv("usrs_need_to_drop.csv", header= False, index= False)

# -------------------------------------------------------------------------------------------

## Drop some users from global table

In [4]:
df_gl = pd.read_csv('all_users_info.csv')
df_gl

,user_id,periods_count,first_plan_id,last_plan_id,bool_changed_plan,first_app_id,last_app_id,first_platform_id,last_platform_id,last_subs_status,...,user_had_trial,total_profit,bool_autorenew_off,count_course_activity_id,median_cource_grade_id,count_content_id,kids_amount,total_usage_steps,bool_isMember_of_campaign,bool_is_school
0,439,4,46,46,False,1.0,1.0,2,2,2,...,False,42246,0,234.0,1.0,234.0,1.0,78.0,0.0,1.0
1,4300,2,46,46,False,1.0,1.0,2,2,2,...,False,20148,1,-1.0,-1.0,-1.0,1.0,4.0,0.0,1.0
2,4534,4,126,126,False,5.0,5.0,3,3,2,...,False,19296,0,462.0,1.0,462.0,2.0,207.0,0.0,1.0
3,8207,4,112,112,False,5.0,5.0,3,3,2,...,False,19296,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,45211,2,221,221,False,5.0,5.0,3,3,1,...,True,9484,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,2260070,1,33,33,False,2.0,2.0,2,2,1,...,False,559,0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
3119,2260091,1,33,33,False,2.0,2.0,2,2,1,...,False,559,0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
3120,2260130,1,33,33,False,2.0,2.0,2,2,1,...,False,559,1,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
3121,2260147,1,34,34,False,2.0,2.0,2,2,1,...,False,3499,0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0


In [13]:
df_gl = df_gl.drop(df_gl[np.isin(usrs_need_to_drop, df_gl.user_id.values)].index)
df_gl

ValueError: Item wrong length 5899 instead of 3123.